In [ ]:
#final code for capturing discriminative features to be written here

In [1]:
#import all file in this block
import gensim
from gensim.models import Word2Vec
from pprint import pprint
import math
import numpy as np


#tests
from sklearn.neural_network import MLPClassifier

In [ ]:
#declaration of global variables

In [2]:
#readDataFiles : reads the file and return list of list of words, and label if necessary
#path_var: variable for path of file to be read, mode: type of data(test - 1 or trian - 0)
def readDataFiles(path_var, mode):
    data = []
    label = []
    with open(path_var , 'r') as file_obj:
        for line in file_obj:
            temp = line.split(',')
            data.append(temp[:3])
            if(1 != mode):
                label.append(temp[-1][0]) #temp[-1][0] since the label is using single digit for representation
    if(1 != mode):
        return data, label
    else:
        return data

In [3]:
#generateModel : generates the word2vec model for the data gathered
def generateModel(train_set, val_set, test_set):
    word_set = train_set + val_set + test_set
    word_model = gensim.models.Word2Vec(word_set, min_count = 1, size = 30, window = 5)
    return word_model

In [4]:
def getEucledianDistanceVector(data, word_model, fv_size):
    n = len(data)
    X = np.matrix(np.zeros((n, fv_size)))
    i = 0
    for i in range(n):
        v = np.zeros(fv_size)
        v[0] = np.linalg.norm(word_model[data[i][0]]-word_model[data[i][2]])
        v[1] = np.linalg.norm(word_model[data[i][1]]-word_model[data[i][2]])
#        print(v[0],v[1])
        X[i, : ] = v
#        print ("   ", X[i])
        i += 1
    return X

In [5]:
def getCosineVector(data, word_model, fv_size):
    n = len(data)
    X = np.matrix(np.zeros((n, fv_size)))
#    print(X.shape)
    i = 0
    for i in range(n):
        v = np.zeros(fv_size)
        v[0] = word_model.similarity(data[i][0], data[i][2])
        v[1] = word_model.similarity(data[i][1], data[i][2])
#        print(v[0],v[1])
        X[i, : ] = v
#        print ("   ", X[i])
        i += 1
    return X

In [6]:
#class 0 is represented by index 0 and 1 by index 1 in rows of Y matrix 
def getClassVector(data, no_class):
    n = len(data)
    Y = np.matrix(np.zeros((n, no_class)))
    i = 0
    for i in range(n):
        Y[i, int(data[i])] = 1
    return Y

In [7]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [8]:
def logisticRegression(train_X, train_Y, epochs, learning_rate):
    size, m = train_X.shape
    train_X = np.hstack((train_X, np.ones((size, 1))))
    m, n = (train_X.shape[1], train_Y.shape[1])
    w = np.random.rand(m, n)
    w = w / 1000
    for epoch in range(epochs):
        print (epoch, end = " ")
        for index in range(size):
            inp = train_X[index]
            target = train_Y[index]
            op = np.matmul(inp, w)
            op = sigmoid(op)
            diff = op - target
            update = np.matmul(inp.T, diff)
            w = w - learning_rate * update
    print()
    return w

In [9]:
def mlpClassifier(train_X, train_Y, epochs, lr):
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1, learning_rate='constant', learning_rate_init=lr ,max_iter=epochs)
    clf.fit(train_X, train_Y)
    return clf

In [10]:
def correct_tag(target):
    return np.argmax(target)

In [22]:
def train(train_X, train_Y, model, epochs, learning_rate):
    if(0 == model):
        w = logisticRegression(train_X, train_Y, epochs, learning_rate)
    elif(1==model):
        l = len(train_Y)
        Y = np.zeros(l)
        i = 0
        for i in range(l):
            Y[i] = correct_tag(train_Y[i])
        w = mlpClassifier(train_X, Y, epochs, learning_rate)
    return w

In [20]:
def predict(inp , w):
    op = np.matmul(inp, w)
    return np.argmax(op)

In [51]:
def recall(data_X, data_Y, w, model):
    TP=0
    TOT = 0
    size = len(data_X)
    if(0==model):
        for index in range(size):
            inp = np.matrix(np.hstack((data_X[index], np.ones((1, 1)))))
            target = data_Y[index]
            if(1 == correct_tag(target)):
                TOT+=1
                if(1 == predict(inp,w)):
                    TP+=1
    elif(1==model):
        pr = w.predict(data_X)
        for index in range(size):
            target = data_Y[index]
            if(1 == correct_tag(target)):
                TOT+=1
                if(1 == pr[index]):
                    TP +=1
    return TP/TOT

In [48]:
def precision(data_X, data_Y, w, model):
    TP=0
    TOT = 0
    size = len(data_X)
    if(0==model):
        for index in range(size):
            inp = np.matrix(np.hstack((data_X[index], np.ones((1, 1)))))
            target = data_Y[index]
            if(1 == predict(inp,w)):
                TOT += 1
                if(1 == correct_tag(target)):
                    TP +=1
    elif(1==model):
        pr = w.predict(data_X)
        for index in range(size):
            target = data_Y[index]
            if(1==pr[index]):
                TOT += 1
                if(1 == correct_tag(target)):
                    TP +=1
    return TP/TOT

In [17]:
def accuracy(data_X, data_Y, w, model):
    i = 0
    correct = 0
    size = len(data_X)
    if (0==model):
        for index in range(size):
            inp = np.matrix(np.hstack((data_X[index], np.ones((1, 1)))))
            target = data_Y[index]
            if predict(inp , w) == correct_tag(target):
                correct += 1
            i += 1
    elif(1==model):
        pr = w.predict(data_X)
        for index in range(size):
            target = data_Y[index]
            if pr[i] == correct_tag(target):
                correct += 1
            i += 1
    return (correct / i)*100

In [58]:
def f1_score(prec,rec):
    f = 2*prec*rec
    f = f/(prec+rec)
    return f

In [70]:
# main function
def main():
    
    #setting of path to take data
    train_path = "./training/train.txt"
    val_path = "./training/validation.txt"
    test_path = "./test/test_triples.txt"
    
    #reading data from the respective files
    train_set, label_train = readDataFiles(train_path, 0)
    val_set, label_val = readDataFiles(val_path, 0)
    test_set = readDataFiles(test_path, 1)
    
    #generating word2vec model for the data
    word_model = generateModel(train_set, val_set, test_set)
    
#    print(word_model[train_set[0][0]])
    
#    return
    #generating feature vector to use in the classifier. uses cosine similarity for feature vector
    fv_size = 2
    #train_X = getCosineVector(train_set, word_model, fv_size)
    train_X = getEucledianDistanceVector(train_set, word_model, fv_size)
    
    #generating training label vectors depending on class
    no_class = 2
    train_Y = getClassVector(label_train, no_class)
    
    #training the model usning the vector consisting of cosine similarity values
    model = 1  #0 means logistic regression and 1 means mlp
    epochs = 100
    learning_rate = 0.03
    w = train(train_X, train_Y, model, epochs, learning_rate)
#    pprint (w)
    
    #validating the weight_vector on val matrix
    #val_X = getCosineVector(val_set, word_model, fv_size)
    val_X = getEucledianDistanceVector(val_set, word_model, fv_size)
    val_Y = getClassVector(label_val, no_class)
#    aq = validate(val_X, val_Y, w, model)
#    print(aq)
#    pr = w.predict(val_X)
#    print(pr)
#    pprint(val_Y)
#    aq = validate(val_V,val_Y,w,model)
    acc = accuracy(val_X, val_Y, w, model)
    print("accuracy\t=\t", acc,"%")
    prec = precision(val_X, val_Y, w, model)
    print("precision\t=\t",prec)
    rec = recall(val_X, val_Y, w, model)
    print("recall\t\t=\t",rec)
    f1 = f1_score(prec,rec)
    print("f1_score\t=\t",f1)
    
    


In [71]:
if __name__ == '__main__':
    main()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


accuracy	=	 48.163115356355625 %
precision	=	 0.4899615548910722
recall		=	 0.8409090909090909
f1_score	=	 0.6191632928475034


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
